#**2° PARCIAL - MapReduce BOW**
---
```
Universidad Nacional de San Antonio Abad del Cusco
Asignatura: Mineria de Datos
Docente   : Carlos Fernando Montoya Cubas
Autor     : Etson Ronaldao Rojas Cahuana
Fecha     : 09/01/2022
Lugar     : Cusco, Perú
Proposito : Aplicar MapReduce BOW.
```
---




#Instalacion de pySpark en Google Colaboratory

In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9

     |████████████████████████████████| 204.2 MB 36 kB/s 
     |████████████████████████████████| 198 kB 23.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=68a09569485e0a1600f81eb9dbb616b1c137fe3539e46cd12bd67f74f36e499b
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


#Inicializar variables de entorno

In [3]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

#Mostrar version de entornos spark

In [4]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

#**Bag Of Words (Bow)**

Es un método que se utiliza en el procesado del lenguaje para representar documentos ignorando el orden de las palabras. En este modelo, cada documento parece una bolsa que contiene algunas palabras. Este método permite un modelado de las palabras basado en diccionarios, donde cada bolsa contiene unas cuantas palabras del diccionario. Las principales ventajas de utilizar este modelo es su facilidad de uso y su eficiencia computacional.







In [14]:
import math
data=[(1,'Los titanes no son nuestros únicos enemigos'),
      (2,"Nadie puede predecir el resultado, cada decisión que tomas tiene un significado sólo para afectar tu próxima decisión"),
      (3,"Parecía que tenias un sueño profundo, pero parece que sigues soñando despierto")]
#creacion para el tokenizer o bag of words
lines=sc.parallelize(data)
lines.collect()
map1=lines.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])
map1.collect()
#utilizacion del map reduce
reduce=map1.reduceByKey(lambda x,y:x+y)
#imprimir el baw of word de las 3 oraciones
reduce.collect()

[((1, 'Los'), 1),
 ((1, 'nuestros'), 1),
 ((1, 'únicos'), 1),
 ((1, 'enemigos'), 1),
 ((2, 'predecir'), 1),
 ((2, 'el'), 1),
 ((2, 'cada'), 1),
 ((2, 'decisión'), 2),
 ((2, 'que'), 1),
 ((2, 'tomas'), 1),
 ((2, 'significado'), 1),
 ((2, 'para'), 1),
 ((2, 'próxima'), 1),
 ((3, 'Parecía'), 1),
 ((3, 'un'), 1),
 ((3, 'profundo,'), 1),
 ((3, 'sigues'), 1),
 ((3, 'despierto'), 1),
 ((1, 'titanes'), 1),
 ((1, 'no'), 1),
 ((1, 'son'), 1),
 ((2, 'Nadie'), 1),
 ((2, 'puede'), 1),
 ((2, 'resultado,'), 1),
 ((2, 'tiene'), 1),
 ((2, 'un'), 1),
 ((2, 'sólo'), 1),
 ((2, 'afectar'), 1),
 ((2, 'tu'), 1),
 ((3, 'que'), 2),
 ((3, 'tenias'), 1),
 ((3, 'sueño'), 1),
 ((3, 'pero'), 1),
 ((3, 'parece'), 1),
 ((3, 'soñando'), 1)]

#**Aplicacion TF- IDF**

In [15]:
#Calculamos TF
tf=reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))
tf.collect()
map3=reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
map3.collect()
map4=map3.map(lambda x:(x[0],x[1][2]))
map4.collect()
reduce2=map4.reduceByKey(lambda x,y:x+y)
reduce2.collect()
#Calculamos IDF
idf=reduce2.map(lambda x: (x[0],math.log10(len(data)/x[1])))
idf.collect()
rdd=tf.join(idf)
rdd.collect()
#calculamos TF-IDF
rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()

#**Mostrar Tabla TF-IDF**

In [16]:
park = SparkSession(sc)
rdd=rdd.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2]))
hasattr(rdd, "toDF")
rdd.toDF(["ID","Token","Bag_of_Word","TF-IDF"]).show(50)

+---+-----------+-----------+-------------------+
| ID|      Token|Bag_of_Word|             TF-IDF|
+---+-----------+-----------+-------------------+
|  1|    titanes|          1|0.47712125471966244|
|  1|         no|          1|0.47712125471966244|
|  1|        son|          1|0.47712125471966244|
|  1|        Los|          1|0.47712125471966244|
|  1|   nuestros|          1|0.47712125471966244|
|  1|     únicos|          1|0.47712125471966244|
|  1|   enemigos|          1|0.47712125471966244|
|  2|   predecir|          1|0.47712125471966244|
|  2|         el|          1|0.47712125471966244|
|  2|      tomas|          1|0.47712125471966244|
|  2|significado|          1|0.47712125471966244|
|  2|      puede|          1|0.47712125471966244|
|  2| resultado,|          1|0.47712125471966244|
|  2|      tiene|          1|0.47712125471966244|
|  2|         tu|          1|0.47712125471966244|
|  2|       cada|          1|0.47712125471966244|
|  2|   decisión|          2|0.47712125471966244|
